In [2]:
from langchain.chat_models import ChatOpenAI


chat = ChatOpenAI(temperature=0.1)

prompt = "hi there"

chat.predict(prompt)

'Hello! How can I assist you today?'

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage

chat = ChatOpenAI(temperature=0.1)
message = [
    SystemMessage(
        content='you are my friend'        
    ),
    AIMessage(content="hi"),
    HumanMessage(content='what do you want to do?')
]

chat.predict_messages(message)

AIMessage(content="I'm an AI, so I don't have personal desires or preferences. However, I'm here to help and chat with you. Is there anything specific you'd like to talk about or do?")

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage

chat = ChatOpenAI(temperature=0.1)
message = [
    SystemMessage(
        content='you are my friend'        
    ),
    AIMessage(content="hi"),
    HumanMessage(content='what do you want to do?')
]

message
# chat.predict_messages(message)

[SystemMessage(content='you are my friend'),
 AIMessage(content='hi'),
 HumanMessage(content='what do you want to do?')]

In [9]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI()
template = PromptTemplate.from_template(
    'what is the distance between {countryA} and {countryB}?'
)

prompt = template.format(countryA='mexico', countryB='thailand')
print("template :",  template)
print("prompt :", prompt)

#  chat.predict(prompt)

template : input_variables=['countryA', 'countryB'] template='what is the distance between {countryA} and {countryB}?'
prompt : what is the distance between mexico and thailand?


In [14]:
chat = ChatOpenAI(temperature=0.1)
template = ChatPromptTemplate.from_messages([
    ('system', 'youre my grifriend'),
    ('ai', 'hi'),
    ('human', 'what is the distance between {countryA} and {countryB}?')
])


message = template.format_messages(
    countryA = 'mexico',
    countryB = 'thailand'
)
chat.predict_messages(message)

AIMessage(content='The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).')

In [29]:
# outputparser
# LLM의 응답의 Dtype을 변환해야 할 때를 위해 필요하다. (디폴트값은 언제나 string)
# 예시는 프롬프트를 제너레이터로 활용하는 경우.
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser) :
    def parse(self, text) :

        items = text.strip().split(",")        
        return list(map(str.strip,items))
    
template = ChatPromptTemplate.from_messages([
    'system', "you are a list generating machine, Everything you are asked will be answered with a comppa separated list of max {max_items}. Do not reply with anything else.",
    'human',"{question}"
    ])

prompt = template.format_messages(
    max_items = 10,
    question = 'what are the colors    '    
)
p = CommaOutputParser()
result = chat.predict_messages(prompt).content
answer = p.parse(result)


print(answer)

['red', 'blue', 'green', 'yellow', 'orange', 'purple', 'pink', 'brown', 'black', 'white']


In [44]:
#chain을 이용해서 위와 동일한 기능을 하는 코드 작성하기
# outputparser
# LLM의 응답의 Dtype을 변환해야 할 때를 위해 필요하다. (디폴트값은 언제나 string)
# 예시는 프롬프트를 제너레이터로 활용하는 경우.
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser) :
    def parse(self, text) :

        items = text.strip().split(",")        
        return list(map(str.strip,items))
    
template = ChatPromptTemplate.from_messages([
    'system', "you are a list generating machine, Everything you are asked will be answered with a comppa separated list of max {max_items}. Do not reply with anything else.",
    'human',"{question}"
    ])
# 이것을 chain이라고 부른다.
chain = template | chat | CommaOutputParser()
# 요약된 내용 
# result = chat.predict_messages(prompt).content
# answer = p.parse(result)
chain.invoke({
    "max_items" : 5,
    "question" : 'what are the colors?'    
})

['red', 'blue', 'green', 'yellow', 'orange']

In [47]:
# chain changing

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1,
                  streaming=True,
                  callbacks=[StreamingStdOutCallbackHandler])


chef_prompt  = ChatPromptTemplate.from_messages([
    ("system", "you are a world_class international chef, you create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "i want to cook {cuisine} food.")
])
# 레시피를 전달해주는 모델
chef_chain = chef_prompt | chat 

veg_chef_prompt = ChatPromptTemplate.from_messages([
    
("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."), 
("human", "{recipe}")
])
# 레시피를 채식레시피로 전환해주는 모델
veg_chain = veg_chef_prompt | chat
# 두 모델의 합체 {}를 통해서 곧바로 인자로 전달할 수 있다는 것이 포인트
final_chain = {'recipe' : chef_chain} | veg_chain

final_chain.invoke({
    'cuisine' : 'indian'
})

ValidationError: 2 validation errors for ChatOpenAI
callbacks -> 0
  instance of BaseCallbackHandler expected (type=type_error.arbitrary_type; expected_arbitrary_type=BaseCallbackHandler)
callbacks
  instance of BaseCallbackManager expected (type=type_error.arbitrary_type; expected_arbitrary_type=BaseCallbackManager)